This example code will download a 5 minute high frequency hydrophone clip in mseed file format from ocean observatories raw data repository or load a previously downloaded clip and then use obsspy to plot it and save it as a wave file 

To run this in Anaconda you may need to load an extra library if not already loaded. The first is obspy (obs for ocean bottom seismometers) as this knows how to load mseed files - a standard in seismic community - and how the high frequency hydrophone data is stored in on the raw data repository. See the note here on how to install the binaries here: https://github.com/obspy/obspy/wiki/Installation-via-Anaconda

This notebook is based on the workbook from Pete Cable - https://github.com/petercable/shared_notebooks/blob/master/antelope/Antelope_to_WAV.ipynb

In [6]:
from obspy import read
from obspy.core import UTCDateTime
import wave
import datetime
import matplotlib
from ipywidgets import widgets
import matplotlib.pyplot as plt
import soundfile as sf
import sounddevice as sd
from IPython.display import Audio

%matplotlib inline
data_url = 'https://rawdata.oceanobservatories.org/files/RS01SLBS/LJ01A/09-HYDBBA102/2017/10/06/OO-HYVM1--YDH-2017-10-06T20:10:00.000015.mseed'
localFileName = '../data/merged_hydrophone.mseed'

You can download directly from OOI in the program via the url or to speed up development download the file and place it in the same directory as this notebook and then read the local file.

In [7]:
loadFromOOI=False

if loadFromOOI==True :
    stream = read(data_url)
else:
    stream = read(localFileName)  # Read Previoulsy Download local file for speed

# print some stats about this signal
stream

1 Trace(s) in Stream:
OO.HYVM2..YDH | 2017-08-21T09:00:00.000000Z - 2017-08-21T11:00:04.450547Z | 64000.0 Hz, 461084836 samples

In [10]:
def getSamplingFreq(s):
    string = str(s).split(" ")
    fs = 0;
    for i in range(0, len(string) - 1) :
        char = string[i] 
        count = 1
        if (char == "|"):
            count = count + 1
        if (count == 2):
            fs = string[i + 1]
    return float(fs)

def playSound(sound):
    sound = sound.copy()
    sound.normalize()
    sound.data = (sound.data * (2**31-1)).astype('int32')
    Fs = getSamplingFreq(sound)
    samplerate = 4*Fs;
    #Audio(data=sound, rate=samplerate)
    sd.play(sound, samplerate)
    
def getSlice(stream, Num):
    dt = UTCDateTime("2017-08-21T09:00:00")
    print(dt)
    st = stream.slice(dt + Num*10, dt + Num*10 + 10) 
    return st

def spec(Num):
    st = getSlice(stream, Num)
    #x=np.array([row[Number] for row in stream])
    fig, ax = plt.subplots()
    pxx, freq, t, cax = ax.specgram(st[0],Fs=64000,noverlap=5, cmap='plasma')
    # c axis
    cbar = fig.colorbar(cax)
    cbar.set_label('Intensity dB')
    ax.axis("tight")
    # y axis   
    ax.set_ylabel('Frequency [kHz]')
    scale = 1e3                     # KHz
    ticks = matplotlib.ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale))
    ax.yaxis.set_major_formatter(ticks)
    # x axis 
    ax.set_xlabel('Time h:mm:ss')
    def timeTicks(x, pos):
        d = UTCDateTime("2017-08-21T09:00:00").second+Num*30+x
        #d = datetime.timedelta(seconds=x)
        return str(d)
    formatter = matplotlib.ticker.FuncFormatter(timeTicks)
    ax.xaxis.set_major_formatter(formatter)
    # shows spectrogram
    plt.show()
    # time plot
    st.plot()
    # plays recording
    playSound(st[0])
    
widgets.interact(spec, Num=widgets.IntSlider(min=0,max=10,value=0,step =1,continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='Num', max=10), Output()), _dom_classes=('widget-interact',))

<function __main__.spec>

In [13]:
stream[0].stats

         network: OO
         station: HYVM2
        location: 
         channel: YDH
       starttime: 2017-08-21T09:00:00.000000Z
         endtime: 2017-08-21T11:00:04.450547Z
   sampling_rate: 64000.0
           delta: 1.5625e-05
            npts: 461084836
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 237553, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 4096, 'filesize': 973017088})